In [1]:
import numpy as np
import pandas as pd
from collections import Counter

In [2]:
!head -n 3 ./magic04.data

28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.011,-8.2027,40.092,81.8828,g
31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.261,g
162.052,136.031,4.0612,0.0374,0.0187,116.741,-64.858,-45.216,76.96,256.788,g


In [3]:
!cat ./magic04.names

1. Title of Database: MAGIC gamma telescope data 2004

2. Sources:

   (a) Original owner of the database:

       R. K. Bock
       Major Atmospheric Gamma Imaging Cherenkov Telescope project (MAGIC)
       http://wwwmagic.mppmu.mpg.de
       rkb@mail.cern.ch

   (b) Donor:

       P. Savicky
       Institute of Computer Science, AS of CR
       Czech Republic
       savicky@cs.cas.cz

   (c) Date received: May 2007

3. Past Usage:

   (a) Bock, R.K., Chilingarian, A., Gaug, M., Hakl, F., Hengstebeck, T.,
       Jirina, M., Klaschka, J., Kotrc, E., Savicky, P., Towers, S.,
       Vaicilius, A., Wittek W. (2004).
       Methods for multidimensional event classification: a case study
       using images from a Cherenkov gamma-ray telescope.
       Nucl.Instr.Meth. A, 516, pp. 511-528.

   (b) P. Savicky, E. Kotrc.
       Experimental Study of Leaf Confidences for Random Forest.
       Proceedings of COMPSTAT 2004, In: Computational Statistics.
       (Ed.: Antoch J.) - Heidelberg, Physi

In [4]:
columnNames = ['fLength', 'fWidth', 'fSize', 'fConc', 'fConc1', 'fAsym', 'fM3Long', 'fM3Trans', 'fAlpha', 'fDist', 'classes']

In [5]:
energy = pd.read_csv("magic04.data", sep=",", names=columnNames, header=None)

In [6]:
energy.describe()

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist
count,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000
mean,53.250154,22.180966,2.825017,0.380327,0.214657,-4.331745,10.545545,0.249726,27.645707,193.818026
std,42.364855,18.346056,0.472599,0.182813,0.110511,59.206062,51.000118,20.827439,26.103621,74.731787
min,4.283500,0.000000,1.941300,0.013100,0.000300,-457.916100,-331.780000,-205.894700,0.000000,1.282600
25%,24.336000,11.863800,2.477100,0.235800,0.128475,-20.586550,-12.842775,-10.849375,5.547925,142.492250
50%,37.147700,17.139900,2.739600,0.354150,0.196500,4.013050,15.314100,0.666200,17.679500,191.851450
75%,70.122175,24.739475,3.101600,0.503700,0.285225,24.063700,35.837800,10.946425,45.883550,240.563825
max,334.177000,256.382000,5.323300,0.893000,0.675200,575.240700,238.321000,179.851000,90.000000,495.561000


In [7]:
def check_purity(dataset):
    column = dataset.iloc[:,-1]
    unique_values = np.unique(column)
    if (len(unique_values) == 1):
        return True
    else:
        return False

for checking if data in a dataset is pure (only contains class variables 'g' or 'h')

In [8]:
def classify_data(dataset):
    column = dataset.iloc[:,-1]
    classes, counts = np.unique(column, return_counts=True)
    index = counts.argmax()
    classification = classes[index]
    
    return classification

classifying all of the dataset['classes'] as either 'g' or 'h' depending on frequency of each variable in the datasetda

In [9]:
def get_potential_splits(dataset):
    potential_splits = {}
    columns = dataset.shape[1]
    for column_index in range(columns - 1):
        potential_splits[column_index] = []
        values = dataset.iloc[:, column_index]
        unique_values = np.unique(values)                #get unique values as potential splits
        
        index = len(unique_values)-1
        unique_values = np.delete(unique_values, index)  #delete last element so split_data() method will not give empty
                                                         #dataset for data_above (data_above > unique_value where
                                                         #unique_value is column.max() will result in data_above being empty)
        potential_splits[column_index] = unique_values
        
    return potential_splits

every unique value in each of the dataset's columns are potential splits.

e.g. fLength's minimum value can be a potential splitting point where data_below will be the rows where dataset['fLength'] <= fLength's minimum value and the data_above will be dataset['fLength'] > fLength's minimum value

we hold the potential splits in a dictionary of lists for further processing later

In [10]:
def split_data(dataset, split_column, split_value):
    split_column_values = dataset.iloc[:, split_column]
    
    data_below = dataset[split_column_values <= split_value]
    data_above = dataset[split_column_values > split_value]
    
    return data_below, data_above

used in conjunction with the potential splits later to determine which split will result in the best metric gain/loss depending on the tree splitting metric (information gain, gain ratio, variance, gini index)

In [11]:
def calculate_entropy(dataset):
    class_column = dataset.iloc[:,-1]
    probabilities = class_column.value_counts(normalize=True)
    entropy = sum(probabilities * -np.log2(probabilities))
    return entropy

In [12]:
def calculate_gini_index(dataset):
    class_column = dataset.iloc[:,-1]
    probabilities = class_column.value_counts(normalize=True)
    gini = 1 - sum(probabilities**2)
    return gini

In [13]:
def calculate_variance(dataset):
    class_column = dataset.iloc[:,-1]
    probabilities = class_column.value_counts()
    unique = np.unique(class_column.values)
    total_rows = len(class_column)
    
    if (len(unique) == 1):
        if(unique[0] == 'g' ):
            gCount = probabilities[0]
            hCount = 0
        else:
            hCount = probabilities[0]
            gCount = 0
    elif (unique[0] == 'g'):
        gCount = probabilities[0]
        hCount = probabilities[1]
    
    mean = (gCount)/total_rows
    #let g = 1 and h = 0
    variance = ((gCount*(1-mean)**2) + (hCount*(0-mean)**2)) / total_rows
    return variance

In [14]:
def calculate_weighted_average(metric, data_below, data_above):
    total_rows = len(data_below) + len(data_above)
    prob_data_below = len(data_below) / total_rows
    prob_data_above = len(data_above) / total_rows
    
    weighted_average = (prob_data_below * metric(data_below)
                        +
                        prob_data_above * metric(data_above))
    
    return weighted_average

In [15]:
def determine_best_split_InfoGain(dataset, potential_splits):
    metric = calculate_entropy
    largest_InfoGain = 0
    
    parent_entropy = calculate_entropy(dataset)
    for index in potential_splits:
        for value in potential_splits[index]:
            data_below, data_above = split_data(dataset, index, value)            
            current_overall_entropy = calculate_weighted_average(metric, data_below, data_above)
            infoGain = parent_entropy - current_overall_entropy
            
            if (infoGain > largest_InfoGain):
                largest_InfoGain = infoGain   #assign new largest info gain for subsequent comparisons
                best_split_column = index
                best_split_value = value
    
    return best_split_column, best_split_value

using potential_splits and split_data from the previous methods, we split_data at every potential split and calculate entropy to get infoGain. save the largest infoGain value in largest_InfoGain variable. at the end of iterating potential_splits, largest_InfoGain will be the split in data which results in the largest InfoGain. we return the column and value of that split.

Similar implementation for the other determine_best_split_metric() methods

In [16]:
def determine_best_split_GainRatio(dataset, potential_splits):
    metric = calculate_entropy
    largest_GainRatio = 0
    
    parent_entropy = calculate_entropy(dataset)
    
    for index in potential_splits:
        for value in potential_splits[index]:
            data_below, data_above = split_data(dataset, index, value)            
            current_overall_entropy = calculate_weighted_average(metric, data_below, data_above)
            infoGain = parent_entropy - current_overall_entropy
            gainRatio = infoGain / parent_entropy
            
            if (gainRatio > largest_GainRatio):
                largest_GainRatio = gainRatio
                best_split_column = index
                best_split_value = value            
            
    return best_split_column, best_split_value

In [17]:
def determine_best_split_variance(dataset, potential_splits):
    metric = calculate_variance
    smallest_variance = 999
    
    parentVariance = calculate_variance(dataset)
    for index in potential_splits:
        for value in potential_splits[index]:
            data_below, data_above = split_data(dataset, index, value)            
            current_overall_var = calculate_weighted_average(metric, data_below, data_above)
            varReduction = parentVariance - current_overall_var
            #smaller variance in data = data becoming less impure
            
            if (varReduction < smallest_variance):
                smallest_variance = varReduction   #assign new smallest variance for subsequent comparisons
                best_split_column = index
                best_split_value = value
    
    return best_split_column, best_split_value

In [18]:
def determine_best_split_GiniIndex(dataset, potential_splits):
    metric = calculate_gini_index
    largest_GiniIndex = 0
    
    parentGini = calculate_gini_index(dataset)
    for index in potential_splits:
        for value in potential_splits[index]:
            data_below, data_above = split_data(dataset, index, value)            
            current_overall_gini = calculate_weighted_average(metric, data_below, data_above)
            giniReduction = parentGini - current_overall_gini
            #larger giniReduction = larger reduction in gini impurity
            
            if (giniReduction > largest_GiniIndex):
                largest_GiniIndex = giniReduction   #assign new largest info gain for subsequent comparisons
                best_split_column = index
                best_split_value = value
    
    return best_split_column, best_split_value

In [39]:
def decision_tree_InfoGain(dataset, counter=0, max_depth=10): #min_samples=5, max_depth=5):
    
    if counter == 0:   #initial execution
        global COLUMN_HEADERS
        COLUMN_HEADERS = dataset.columns
        data = pd.DataFrame(data=dataset)
    else:              #recursive execution
        data = dataset
    
    #base case
    if (check_purity(data)) or (counter == max_depth): #(len(data) < min_samples)
        classification = classify_data(data)
        return classification
    
    #recursive portion
    else:
        counter += 1
        
        #auxiliary functions
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split_InfoGain(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        #grow sub-tree
        feature_name = COLUMN_HEADERS[split_column]
        split_criteria = "{} <= {}".format(feature_name, split_value)
        sub_tree = {split_criteria: []}
        
        yes_branch = decision_tree_InfoGain(data_below, counter)#, min_samples, max_depth)
        no_branch = decision_tree_InfoGain(data_above, counter)#, min_samples, max_depth)
        
        if (yes_branch == no_branch):
            sub_tree = yes_branch
        else:
            sub_tree[split_criteria].append(yes_branch)
            sub_tree[split_criteria].append(no_branch)
        
        return sub_tree

in base case, if data is pure, we can immediately classify it. if not pure yet, we will continue growing the decision tree in the recursive portion.
first, get the potential_splits of the data, then determine_best_split_metric.

the split_criteria will be a string known as:
the feature returned by determine_best_split_metric, feature_name, <= operator, and split_value.
we put this in a list inside a dictionary called sub_tree.

using the splitted data, we recursively call the decision_tree_metric method for each splitted data, data_below and data_above.
the segmented data will then check it's purity, and if not pure, get it's potential splits, determine it's best split, and getting more splitting criterias to grow the tree.

once the data_below or data_above is pure, it will be classified by the classify_data() method mentioned previously, resulting in sub_tree = the classified alphabet ('g' or 'h'). If the yes_branch and no_branch result in the same sub_tree, sub_tree will denote the classified alphabet (this is the very end of the tree). If not, then the sub_tree will be appended by the splitting_criteria.

the method call will result in a the decision tree.

In [40]:
def decision_tree_GainRatio(dataset, counter=0, max_depth=10): #min_samples=10, max_depth=10):
    
    if counter == 0:   #initial execution
        global COLUMN_HEADERS
        COLUMN_HEADERS = dataset.columns
        data = pd.DataFrame(data=dataset)
    else:              #recursive execution
        data = dataset
    
    #base case
    if (check_purity(data)) or (counter == max_depth):
        classification = classify_data(data)
        return classification
    
    #recursive portion
    else:        
        counter += 1
        
        #auxiliary functions
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split_GainRatio(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        #grow sub-tree
        feature_name = COLUMN_HEADERS[split_column]
        split_criteria = "{} <= {}".format(feature_name, split_value)
        sub_tree = {split_criteria: []}
        
        yes_branch = decision_tree_GainRatio(data_below, counter)
        no_branch = decision_tree_GainRatio(data_above, counter)
        
        if (yes_branch == no_branch):
            sub_tree = yes_branch
        else:
            sub_tree[split_criteria].append(yes_branch)
            sub_tree[split_criteria].append(no_branch)
        
        return sub_tree

In [36]:
def decision_tree_variance(dataset, counter=0, max_depth=10):
    
    if counter == 0:   #initial execution
        global COLUMN_HEADERS
        COLUMN_HEADERS = dataset.columns
        data = pd.DataFrame(data=dataset)
    else:              #recursive execution
        data = dataset
    
    #base case
    if (check_purity(data)) or (counter == max_depth):
        classification = classify_data(data)
        return classification
    
    #recursive portion
    else:
        counter += 1
        
        #auxiliary functions
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split_variance(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        #grow sub-tree
        feature_name = COLUMN_HEADERS[split_column]
        split_criteria = "{} <= {}".format(feature_name, split_value)
        sub_tree = {split_criteria: []}
        
        yes_branch = decision_tree_variance(data_below, counter)
        no_branch = decision_tree_variance(data_above, counter)
        
        if (yes_branch == no_branch):
            sub_tree = yes_branch
        else:
            sub_tree[split_criteria].append(yes_branch)
            sub_tree[split_criteria].append(no_branch)
        
        return sub_tree

In [41]:
def decision_tree_GiniIndex(dataset, counter=0, max_depth=10):#, min_samples=10, max_depth=10):
    
    if counter == 0:   #initial execution
        global COLUMN_HEADERS
        COLUMN_HEADERS = dataset.columns
        data = pd.DataFrame(data=dataset)
    else:              #recursive execution
        data = dataset
    
    #base case
    if (check_purity(data)) or (counter == max_depth):  #or (len(data) < min_samples)
        classification = classify_data(data)
        return classification
    
    #recursive portion
    else:
        counter += 1
        
        #auxiliary functions
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split_GiniIndex(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        #grow sub-tree
        feature_name = COLUMN_HEADERS[split_column]
        split_criteria = "{} <= {}".format(feature_name, split_value)
        sub_tree = {split_criteria: []}
        
        yes_branch = decision_tree_GiniIndex(data_below, counter)#, min_samples, max_depth)
        no_branch = decision_tree_GiniIndex(data_above, counter)#, min_samples, max_depth)
        
        if (yes_branch == no_branch):
            sub_tree = yes_branch
        else:
            sub_tree[split_criteria].append(yes_branch)
            sub_tree[split_criteria].append(no_branch)
        
        return sub_tree

In [23]:
def classifier(example, tree):
    split_criteria = list(tree.keys())[0]
    feature_name, _, value = split_criteria.split()
    
    if example[feature_name] <= float(value):
        answer = tree[split_criteria][0]
    else:
        answer = tree[split_criteria][1]
        
    if not isinstance(answer, dict):
        return answer
    else:
        residual_tree = answer
        return classifier(example, residual_tree)  #traverse tree until answer is a leaf node instead of split_criteria

the decision tree's dictionary key is the split_criteria. we use list to get the split_criteria as a variable to use as an index to get the nested lists the dictionary holds.

the nested lists we get will be the true and false portion of the decision tree's split_criteria.

recursively, we apply the same logic on the true or false portion of the initial split criteria (which depends on the example's data we used for the method). the recursion happens until we reach the end of the tree.

the answer we get is the decision_tree's classification

In [24]:
def get_classification(dataset, tree):
    
    predictions = dataset.apply(classifier, axis=1, args=(tree,))
    predictions = predictions.to_numpy()       
    
    return predictions

get a np.array of predicted 'g' or 'h' values based on the test data

In [25]:
def append_classifier(dataset, tree, columnName):
    
    dataset[columnName] = get_classification(dataset, tree)
        
    return dataset

append the np.array to the test data to compare the class columns with our decision_tree's predictions

In [26]:
def ensemble_classifier(testset, tree1, tree2, tree3):
    
    classify_IG = get_classification(testset, tree1)    
    classify_GR = get_classification(testset, tree2)
    classify_var = get_classification(testset, tree3)
    classify_ensemble = []

    for i in range(len(classify_IG)):
        selection = []
        selection.append(classify_IG[i])
        selection.append(classify_GR[i])
        selection.append(classify_var[i])
        mode = Counter(selection)
        classify_ensemble.append(mode.most_common(1)[0][0])

    return classify_ensemble

using the 3 decision tree classifiers, InfoGainTree, GainRatioTree, and varianceTree, use the above voting function to get the highest frequency of 'g' or 'h' to determine the ensemble's prediction.

In [28]:
from sklearn.model_selection import StratifiedKFold
xTrain = []
xTest = []
yTrain = []
yTest = []

X = energy.iloc[:,:10]
y = energy.iloc[:,10]
skf = StratifiedKFold(n_splits=10)
skf.get_n_splits(X,y)
for train_index, test_index in skf.split(X,y):
    xTrain.append(X.iloc[train_index])
    xTest.append(X.iloc[test_index])
    yTrain.append(y.iloc[train_index])
    yTest.append(y.iloc[test_index])

In [29]:
cross_train = []
cross_test = []
for i in range(len(xTrain)):
    xTrain[i]['classes'] = yTrain[i]
    xTest[i]['classes'] = yTest[i]
    cross_train.append(xTrain[i])
    cross_test.append(xTest[i])

C:\Users\dnkr9\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\dnkr9\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [30]:
#sample from test portion to reduce decision_tree training time
total = len(cross_train[0])
g_normalize,h_normalize = cross_train[0].classes.value_counts(normalize=True)
sample_cross_train = []
for i in range(len(cross_train)):
    temp = pd.concat([cross_train[i][cross_train[i].classes=='g'].sample(int((1/9*total)*g_normalize)), energy[energy.classes=='h'].sample(int((1/9*total)*h_normalize))])
    sample_cross_train.append(temp)

In [31]:
sample_cross_train[0].classes.value_counts()

g    1233
h     668
Name: classes, dtype: int64

In [32]:
cross_test[0].classes.value_counts()

g    1234
h     668
Name: classes, dtype: int64

In [33]:
InfoGainTree = decision_tree_InfoGain(sample_cross_train[0])

In [42]:
InfoGainTree2 = decision_tree_InfoGain(sample_cross_train[1])

In [47]:
for i in range(2,10):
    InfoGainTree = decision_tree_InfoGain(sample_cross_train[i])
    InfoGain_Trees.append(InfoGainTree)

In [ ]:
InfoGain_Trees = []
GainRatio_Trees = []
GiniIndex_Trees = []
variance_Trees = []
ensemble_List = []

Due to lack of time to train my decision tree models, I have not only sampled the training portion of the KFold split, but i am also only using 5 folds instead of 10 to evaluate the signicance in mean errors

In [58]:
for i in range(0,5):
    GainRatioTree = decision_tree_GainRatio(sample_cross_train[i])
    GainRatio_Trees.append(GainRatioTree)

In [62]:
for i in range(0,5):
    GiniIndexTree = decision_tree_GiniIndex(sample_cross_train[i])
    GiniIndex_Trees.append(GiniIndexTree)

In [64]:
for i in range(0,5):
    varTree = decision_tree_variance(sample_cross_train[i])
    variance_Trees.append(varTree)

In [37]:
varTree = decision_tree_variance(sample_cross_train[0])

In [73]:
for i in range(0,5):
    ensemble = ensemble_classifier(cross_test[i], InfoGain_Trees[i], GainRatio_Trees[i], variance_Trees[i])
    ensemble_List.append(ensemble)
    #need to manually append ensemble_predictions[i] to the cross_test[i] dataset so that can do pd.crosstab for error calculation

In [80]:
for i in range(0,5):
    cross_test[i]['Ensemble'] = ensemble_List[i]

C:\Users\dnkr9\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [79]:
for i in range(0,5):
    append_classifier(cross_test[i], GiniIndex_Trees[i], 'GiniIndex')

C:\Users\dnkr9\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [86]:
ensemble_error = []
giniIndex_error = []

GiniIndex:

In [87]:
pd.crosstab(cross_test[0]['classes'], cross_test[0]['GiniIndex'], margins=True)

GiniIndex,g,h,All
classes,,,
g,1058,176,1234
h,192,476,668
All,1250,652,1902


In [88]:
accuracy = (1058 + 476) / 1902
accuracy
error = 1 - accuracy
error
giniIndex_error.append(error) #0.1934

In [95]:
pd.crosstab(cross_test[1]['classes'], cross_test[1]['GiniIndex'], margins=True)

GiniIndex,g,h,All
classes,,,
g,1111,123,1234
h,231,437,668
All,1342,560,1902


In [96]:
accuracy = (1111 + 437) / 1902
accuracy
error = 1 - accuracy
error
giniIndex_error.append(error) #0.1861

In [100]:
pd.crosstab(cross_test[2]['classes'], cross_test[2]['GiniIndex'], margins=True)

GiniIndex,g,h,All
classes,,,
g,1087,146,1233
h,195,474,669
All,1282,620,1902


In [102]:
accuracy = (1087 + 474) / 1902
accuracy
error = 1 - accuracy
error
giniIndex_error.append(error) #0.1793

In [104]:
pd.crosstab(cross_test[3]['classes'], cross_test[3]['GiniIndex'], margins=True)

GiniIndex,g,h,All
classes,,,
g,1094,139,1233
h,196,473,669
All,1290,612,1902


In [106]:
accuracy = (1094 + 473) / 1902
accuracy
error = 1 - accuracy
error
giniIndex_error.append(error) #0.1761

In [107]:
pd.crosstab(cross_test[4]['classes'], cross_test[4]['GiniIndex'], margins=True)

GiniIndex,g,h,All
classes,,,
g,1067,166,1233
h,205,464,669
All,1272,630,1902


In [109]:
accuracy = (1067 + 464) / 1902
accuracy
error = 1 - accuracy
error
giniIndex_error.append(error) #0.1951

Ensemble:

In [110]:
pd.crosstab(cross_test[0]['classes'], cross_test[0]['Ensemble'], margins=True)

Ensemble,g,h,All
classes,,,
g,1096,138,1234
h,203,465,668
All,1299,603,1902


In [112]:
accuracy = (1096 + 465) / 1902
accuracy
error = 1 - accuracy
error
ensemble_error.append(error) #0.1793

In [113]:
pd.crosstab(cross_test[1]['classes'], cross_test[1]['Ensemble'], margins=True)

Ensemble,g,h,All
classes,,,
g,1043,191,1234
h,194,474,668
All,1237,665,1902


In [115]:
accuracy = (1043 + 474) / 1902
accuracy
error = 1 - accuracy
error
ensemble_error.append(error) #0.2024

In [116]:
pd.crosstab(cross_test[2]['classes'], cross_test[2]['Ensemble'], margins=True)

Ensemble,g,h,All
classes,,,
g,1074,159,1233
h,191,478,669
All,1265,637,1902


In [118]:
accuracy = (1074 + 478) / 1902
accuracy
error = 1 - accuracy
error
ensemble_error.append(error) #0.1840

In [119]:
pd.crosstab(cross_test[3]['classes'], cross_test[3]['Ensemble'], margins=True)

Ensemble,g,h,All
classes,,,
g,1080,153,1233
h,185,484,669
All,1265,637,1902


In [121]:
accuracy = (1080 + 484) / 1902
accuracy
error = 1 - accuracy
error
ensemble_error.append(error) #0.1777

In [122]:
pd.crosstab(cross_test[4]['classes'], cross_test[4]['Ensemble'], margins=True)

Ensemble,g,h,All
classes,,,
g,1042,191,1233
h,171,498,669
All,1213,689,1902


In [124]:
accuracy = (1042 + 498) / 1902
accuracy
error = 1 - accuracy
error
ensemble_error.append(error) #0.1903

Determine the statistical significance of the error rate difference between ensemble model(M1) and gini-index model(M2).

H0: M1 and M2 are significantly different.
H1: M1 and M2 are not significantly different.

In [127]:
ensemble_error
giniIndex_error

[0.17928496319663512,
 0.20241850683491058,
 0.18401682439537326,
 0.17770767613038907,
 0.19032597266035756]

In [128]:
ensemble_mean_error = sum(ensemble_error)/len(ensemble_error)
giniIndex_mean_error = sum(giniIndex_error)/len(giniIndex_error)

In [141]:
diffSquaredSum = 0
for i in range(len(ensemble_error)):
    diff = ensemble_error[i] - ensemble_mean_error
    diffSquared = diff**2
    diffSquaredSum += diffSquared
ensemble_variance = diffSquaredSum/len(ensemble_error)

In [143]:
diffSquaredSum = 0
for i in range(len(giniIndex_error)):
    diff = giniIndex_error[i] - giniIndex_mean_error
    diffSquared = diff**2
    diffSquaredSum += diffSquared
giniIndex_variance = diffSquaredSum/len(giniIndex_error)

In [146]:
#var(M1 - M2)
varM1M2 = np.sqrt(((giniIndex_variance / 5) + (ensemble_variance / 5)))

In [147]:
#t-test
t = (giniIndex_mean_error - ensemble_mean_error) / np.sqrt((varM1M2/5))

In [149]:
t

-0.022759379403082952

at 5% significance level, df = 4, critical value = +/-2.776
since t* = -0.022 > critical value(-2.776), t* does not lie in the rejection region.
Hence, H0 is not rejected.

#Post-Pruning Part

In [150]:
def filter_df(df, split_criteria):
    feature, _, value = split_criteria.split()
    left = df[df[feature] <= float(value)]
    right = df[df[feature] > float(value)]
    print(split_criteria)
    print(left.shape)
    print(right.shape)
    return left, right

filter dataset by split_criteria for use in recursive call later

In [151]:
def pruning_results(tree, trainset, pruneset):
    
    print("pruneset: ")
    print(pruneset.classes.value_counts())
    
    if (pruneset.shape[0] == 0):
        leaf = trainset.classes.value_counts().index[0]
        return leaf
    else:
        
        print("bef value_counts()")
        print(trainset.classes.value_counts())
        print("aft value_counts()")
        leaf = trainset.classes.value_counts().index[0] #[0] is the higher count, which means classify_data() will classify all values as index[0]
        errors_leaf = sum(pruneset.classes != leaf)     #pruneset.classes != classified value = error
        print("leaf: {}".format(leaf))
        print("errors_leaf: {}".format(errors_leaf))
        errors_decision_node = sum(pruneset.classes != get_classification(pruneset, tree))
        print("errors_decision_node: {}".format(errors_decision_node))
        
                                                #run prediction on pruneset and compare to get number of decision_node errors
    
        if (errors_leaf <= errors_decision_node):   #return one with less errorss for better overall accuracy
            return leaf
        else:
            return tree

In [152]:
def post_pruning(tree, trainset, pruneset):
    split_criteria = list(tree.keys())[0]
    left, right = tree[split_criteria]
    
    #base case
    if not isinstance(left, dict) and not isinstance(right, dict):
        print("left dict?: {}".format(left))
        print("right dict?: {}".format(right))
        return pruning_results(tree, trainset, pruneset)
    
    #recursive portion - when left and right are still decision nodes
    else:
        print("left filter:")
        train_left, train_right = filter_df(trainset, split_criteria) #filter train by split_criteria
        print("right filter:")
        prune_left, prune_right = filter_df(pruneset, split_criteria) #filter prune by split_criteria
        
        if isinstance(left, dict):
            left = post_pruning(left, train_left, prune_left)     #keep calling post_pruning until we reach leaf nodes
        
        if isinstance(right, dict):
            right = post_pruning(right, train_right, prune_right) #keep calling post_pruning until we reach leaf nodes
        
        tree = {split_criteria: [left, right]}             #once we get back, return the pruning results into the tree
        
        return pruning_results(tree, trainset, pruneset)   #test if leaf or decision nodes have more errors and return accordingly

using recursion to traverse the tree, we decide if we should prune the last split_criteria from the tree by looking at the split_criteria's left and right values.

base case:
if the left and right are not decision nodes(split_criteria), meaning they are leaf nodes('g', 'h') (the result of classify_data()), we look at our train dataset's class distribution and take the higher frequency (because classify_data() classifies all the data as the higher frequency value). this is the leaf node's value. (either 'g' or 'h')

we sum the no. of rows of data where prune dataset's class value is != to the leaf node's value. this is the number of errors of the leaf node's classification.

next, we sum the no. of rows of data where prune dataset's class values != to the decision node's prediction of the prune dataset's class values. we sum the count and this is the number of errors of the decision node.

we compare the no. of errors in the leaf's classification and the no. of errors of the decision node. if there are more errors in the leaf's classification, then we return the decision node. else we reutrn the leaf node.


recursion part:
call post_pruning until we reach leaf nodes for base case scenario to happen.

In [153]:
g_normalize,h_normalize = energy['classes'].value_counts(normalize=True)
total = energy.shape[0]

In [154]:
train = pd.concat([energy[energy.classes=='g'].sample(int((1/3*total)*g_normalize)), energy[energy.classes=='h'].sample(int((1/3*total)*h_normalize))])
subset = energy.loc[~energy.index.isin(train.index)]
total = subset.shape[0]
prune = pd.concat([subset[subset.classes=='g'].sample(int((1/2*total)*g_normalize)), subset[subset.classes=='h'].sample(int((1/2*total)*h_normalize))])
test = subset.loc[~subset.index.isin(prune.index)]

stratified sampling where:
train is 1/3 of the base dataset,

subset is 2/3 of the base subset where train's row indexes are not in it. subset is only used to get prune and test datasets as follows:

prune is 1/2 of the subset (1/2subset * 2/3base = 1/3base)
test is the other 1/2 of subset where prune's row indexes are not in it.

for this portion, i restarted my kernel by mistake and did not have enough time to re-run the decision tree. the code has been tested and is working. running it should work fine albeit a 10minute training time for the GiniIndexTree

In [ ]:
GiniIndexTree = decision_tree_GiniIndex(train)

In [ ]:
Prune_GiniIndex = post_pruning(GiniIndexTree, train, prune)

In [ ]:
append_classifier(test, GiniIndexTree, 'Not_Pruned_GiniIndex')
append_classifier(test, Prune_GiniIndex, 'Pruned_GiniIndex')

In [ ]:
pd.crosstab(test['classes'], test['Pruned_GiniIndex'], margins=True)

Pruned_GiniIndex - TP: 3772, FP: 648. TP Rate = 85.34%, FP Rate = 14.66%, Accuracy = 84.42%

In [ ]:
pd.crosstab(test['classes'], test['Not_Pruned_GiniIndex'], margins=True)

Not_Pruned_GiniIndex - TP: 3608, FP: 661. TP Rate = 84.51%, FP Rate = 15.48%, Accuracy = 81.63%